In [2]:
import os
import shutil

In [185]:
def get_nf_files(folder):
    nf_files = []
    for entry in os.listdir(folder):
        if os.path.isfile(os.path.join(folder, entry)) and entry.endswith('.nf'):
            nf_files.append(os.path.join(folder, entry))
        if os.path.isdir(os.path.join(folder, entry)):
            new_files = get_nf_files(os.path.join(folder, entry))
            for file in new_files:
                nf_files.append(file)
    return nf_files

In [189]:
#with open('BALROG-MON\\modules\\BUSCO.nf') as file:
#    text = file.read().split('\n')
#    print(text[2].split())

In [215]:
def get_workflows(pipeline_dir, workflow_dir):
    nf_workflows = get_nf_files(pipeline+ '/' + workflow_dir)
    print(pipeline_dir + ' contains these workflows: \n')
    for wf in nf_workflows:
        print(wf)
    return nf_workflows

In [217]:
def get_used_files(pipeline_dir, workflows):
    # Creating list to store subworkflows and modules that are used
    used_modules = []
    used_subworkflows = []
    
    for workflow in workflows:
        print("Subworkflows in: " + workflow.split('\\')[-1])
        with open(workflow) as file:
            text = file.read().split('\n')
            for line in text:
                if line.split(' ')[0] == "include":
                    print(line.split(' ')[-1])
                    subworkflow = line.split(' ')[-1][4:]
                    used_subworkflows.append(subworkflow.split('/')[-1][:-1])
    
                    print('modules used in: ' + subworkflow)
                    with open(workflow.split('/')[0] + '/' + subworkflow[:-1]) as file:
                        sw_text = file.read().split('\n')
                        for line in sw_text:
                            if line.split(' ')[0] == "include":
                                print(line.split(' ')[-1])
                                used_modules.append(line.split(' ')[-1].split('/')[-1])
                                
    return [used_subworkflows, used_modules]

In [219]:
def get_unused_files(pipeline_dir, subworkflow_dir, module_dir, used_sub, used_mod):
## Creating list of subworkflows to move to unused
    unused_subworkflows = []
    unused_modules = []
    
    for sub in os.listdir(pipeline_dir + '/' + subworkflow_dir):
        if sub not in used_sub:
            print(sub + ' is not used in this pipeline and will be scheduled for archival')
            unused_subworkflows.append(sub)
    
    print('---------------------------------------------------------------')
    
    for mod in os.listdir(pipeline_dir + '/' + module_dir):
        if mod not in used_mod:
            print(mod + ' is not used in this pipeline and will be scheduled for archival')
            unused_modules.append(mod)

    return [unused_subworkflows, unused_modules]

In [221]:
def create_archive_structure(pipeline_dir): 
    # Define the directory paths
    main_dir = pipeline_dir + '/Archive_NFParser'
    sub_dirs = ["modules", "subworkflows"]
    
    # Create the main directory
    os.makedirs(main_dir, exist_ok=True)
    
    # Create subdirectories inside the main directory
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(main_dir, sub_dir), exist_ok=True)
    
    print("Directory structure created (or already exists).")

In [223]:
def move_file_with_suffix(src_file, dest_dir):
    # Get the base name and extension of the file
    base_name, ext = os.path.splitext(os.path.basename(src_file))
    dest_file = os.path.join(dest_dir, os.path.basename(src_file))
    
    # Check if the file already exists
    counter = 1
    while os.path.exists(dest_file):
        # Modify the file name with a suffix (e.g., -1, -2, etc.)
        dest_file = os.path.join(dest_dir, f"{base_name}-{counter}{ext}")
        counter += 1

    # Move the file
    shutil.move(src_file, dest_file)
    print(f"Moved: {src_file} to {dest_file}")



In [259]:
def archive_files(pipeline_dir, source_dir, file_list, archive_folder):
    if len(file_list) > 0:
        for file in file_list:
            move_file_with_suffix(pipeline_dir + '/' + source_dir + '/' + file, pipeline_dir + '/Archive_NFParser/' + archive_folder)
    else:
        print('There were no unused files in ' + source_dir + ' so no files were moved')

In [261]:
# Args
pipeline_dir = 'BALROG-MON-TEST'
workflow_dir = 'workflows'
subworkflow_dir = 'subworkflows'
module_dir = 'modules'

In [263]:
def whole_process(pipeline_dir, workflow_dir, subworkflow_dir, module_dir):
    workflows = get_workflows(pipeline_dir, workflow_dir)
    used_files = get_used_files(pipeline_dir, workflows)
    unused_files = get_unused_files(pipeline_dir, subworkflow_dir, module_dir, used_files[0], used_files[1])
    create_archive_structure(pipeline_dir)
    archive_files(pipeline_dir, subworkflow_dir, unused_files[0], subworkflow_dir)
    archive_files(pipeline_dir, module_dir, unused_files[1], module_dir)

In [265]:
whole_process(pipeline_dir, workflow_dir, subworkflow_dir, module_dir)

BALROG-MON-TEST contains these workflows: 

BALROG-MON/workflows\ONT_METAGENOMIC.nf
BALROG-MON/workflows\ONT_METAGENOMIC_RNA.nf
BALROG-MON/workflows\SHORT_READ_ISOLATE.nf
BALROG-MON/workflows\SHORT_READ_METAGENOMIC.nf
BALROG-MON/workflows\SHORT_READ_METAGENOMIC_RNA.nf
Subworkflows in: ONT_METAGENOMIC.nf
'../subworkflows/READ_QC_ONT.nf'
modules used in: subworkflows/READ_QC_ONT.nf'
'../modules/FASTQC_ONT.nf'
'../modules/FASTQC_ONT.nf'
'../modules/FASTQC_ONT.nf'
'../modules/PORECHOP.nf'
'../modules/CHOPPER.nf'
'../modules/RENAME_READS.nf'
'../subworkflows/PLASMID_PREDICTION.nf'
modules used in: subworkflows/PLASMID_PREDICTION.nf'
'../modules/PLASMER.nf'
'../modules/PLASMER_SORT.nf'
'../modules/QUAST.nf'
'../modules/QUAST.nf'
'../modules/QUAST.nf'
'../modules/VIRALVERIFY.nf'
'../modules/PLASMER_DB.nf'
'../modules/PFAM_DB.nf'
'../modules/VIRSORTER2.nf'
'../modules/VIRSORTER2_DB.nf'
'../modules/MEF.nf'
'../subworkflows/METAGENOMIC_COMMUNITY_ANALYSIS_ONT.nf'
modules used in: subworkflows/MET

## Geting Containers

In [24]:
import os
import shutil
import pandas as pd

In [26]:
# Args
pipeline_dir = 'BALROG-MON'
module_dir = 'modules'
output_csv = 'test_contaienrs.csv'

In [28]:
def get_nf_files(folder):
    nf_files = []
    for entry in os.listdir(folder):
        if os.path.isfile(os.path.join(folder, entry)) and entry.endswith('.nf'):
            nf_files.append(os.path.join(folder, entry))
        if os.path.isdir(os.path.join(folder, entry)):
            new_files = get_nf_files(os.path.join(folder, entry))
            for file in new_files:
                nf_files.append(file)
    return nf_files

In [40]:
def get_contaienrs(pipeline_dir, module_dir, output_csv):
    
    nf_files = get_nf_files(pipeline_dir + '/' + module_dir)
    nf_files

    paths = []
    conts = []
    
    for path in nf_files:
        with open(path) as file:
            text = file.read().split('\n')
            for line in text:
                words = line.split()
                if 'container' in words:
                    paths.append(path)
                    conts.append(words[1])
    
    conts_df = pd.DataFrame()
    conts_df['path'] = paths
    conts_df['cont'] = conts
    
    conts_df.to_csv('Current_Containers.csv', index=False)